# 20-Tableau - CitiBike

@Author Jeffery Brown (daddyjab)<br>
@Date 4/18/19<br>
@File Citibike_Exploration_Wait_Time_JAB

# Exploratory Analysis and Data Munging

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import datetime as dt

from pprint import pprint


In [2]:
def gen_idle_time_df( a_df ):
# Function to add bike idle time information to the
# CitiBike data that has already been processed to add path information
#
# Arguments:
#    a_df: DataFrame populated with CitiBike w/ Paths data
#
# Returns:
#    retval: DataFrame with bike idle time populated for use in Tableau

    # Note: The CitiBike Paths data has been preprocessed and field names
    # are changed from the official dataset:
    # Bike ID
    # _1=26152,

    # End Station ID,End Station Latitude,End Station Longitude,End Station Name
    # _2=3213, _3=40.71848892, _4=-74.047726625, _5='Van Vorst Park',

    # Path ID,Path Order
    # _6='26152_2019-02-01_09_16_40_3272', _7=1,

    # Start Station ID,Start Station Latitude,Start Station Longitude,Start Station Name
    # _8=3272, _9=40.723331586464354, _10=-74.04595255851744, _11='Jersey & 3rd',

    # Start Time,Stop Time
    # _12='2019-02-01 09:16:40.8280', _13='2019-02-01 09:19:51.1310',

    # Trip Distance (miles),Trip Duration (mins),Trip Speed (mph
    # _14=0.3468923185159586, _15=3.1666666666666665, _16=6.572696561355004,

    # User Age,User Birth Year,User Gender,User Type
    # _17=46, _18=1973, _19='Male', _20='Annual Subscriber')
    


    # If no DataFrame has been provided, then return
    if a_df is None:
        return None
    
    # Sorting by Bike ID then Start time then Stop Time to ensure that
    # the movement of each bike from station to station is in order
    a_df.sort_values(by=['Bike ID','Start Time', 'Stop Time'], inplace=True)
    
    # Reset the indices
    a_df.reset_index(drop=True, inplace=True)

    # Loop through each record and use the Start and Stop Time data to
    # determine how long a bike was idle before each new trip started
    # Note: Idle time for the very first data point will be None

    idle_time_list = []
    bikeid_last = None;
    stoptime_last = None;

    for r in a_df.itertuples():
#         print(f">> Processing Bike ID: {r._1}")
        
        # Get the Start Time as a Timestamp object
        try:
            start_time = pd.to_datetime(r._12)
            
        except ValueError:
            # If an error occurs with Start Time,
            # then skip this entry since idle time cannot be generated
#             print(f">>>> *** Start Time is not valid: start_time: {r._12}")
            start_time = None
        
        # If the Bike ID has changed then set idle time to None,
        # otherwise set it to the time difference since Bike was last used
        
        if (stoptime_last is None) or (start_time is None):
            # Ok, if either of these value is None then cannot calc Idle Time
#             print(f">>>> *** Setting Idle Time to None due to invalid value(s): start_time: {r._12} or stoptime_last: {stoptime_last}")

            idle_time = None
            
        else:
            # Ok, have the times needed to calculate Idle Time
#             print(f">>>> Start Time: {start_time:%a %b %m/%d/%y %I:%M:%S %p} vs. Last Stop Time: {stoptime_last:%a %b %m/%d/%y %I:%M:%S %p}")
            
            if r._1 != bikeid_last:    
                # Starting to use a new bike, so set idle time to None
                idle_time = None
#                 print(f">>>> New Bike: Setting Idle Time to None")


            else:
                # Still using the same bike, so calculate the idle time in minutes
                idle_time = (start_time - stoptime_last).total_seconds() / 60.0
#                 print(f">>>> Same Bike: Idle Time (mins.): {idle_time}")

        # Add this idle time to the list
        idle_time_list.append(idle_time)

        # Save the current Bike ID
        bikeid_last = r._1

        # Save the current Stop Time as a Timestamp object
        try:
            stoptime_last = pd.to_datetime(r._13);
            
        except ValueError:
            stoptime_last = None
#             print(f">> *** ValueError: stoptime_last: {stoptime_last}")

        # DEBUG *******************************************************************************
        # if r.Index > 80: break
        # DEBUG *******************************************************************************

    # Add an "Idle Time" column to the existing DataFrame
    # Note: This assumes that the order is being preserved properly!
#     print( f"idle_time_list # of entries: {len(idle_time_list)}")
#     print( idle_time_list )

    a_df['Idle Time (mins)'] = idle_time_list

    return a_df

In [3]:
# Import CitiBike data
cb_paths_i_file_list = [
        "new-Paths-JC-201902-citibike-tripdata.csv",
        "new-Paths-JC-201901-citibike-tripdata.csv",
        "new-Paths-201902-citibike-tripdata.csv",
        "new-Paths-201901-citibike-tripdata.csv"
         ]

# cb_paths_i_file_list = [
#         "new-Paths-JC-201902-citibike-tripdata.csv"
#          ]

In [4]:
%%time
# Loop through the list of CitiBike data files to process
for cb_p_i_file in cb_paths_i_file_list:
    
    print(" ")
    print(f"*** PROCESSING FILE: { cb_p_i_file } *****************************************")
    
    # Read the CSV file into a DataFrame
    cb_i_df = pd.read_csv(cb_p_i_file, index_col=False)
    
    # Print a progress message
    print(f"> Records in this file: { cb_i_df['Bike ID'].count() }")
    
    # Process path data in the DataFrame
    %time cb_o_df = gen_idle_time_df( cb_i_df )

    # Print a progress message
    try:
        print(f"> Idle Time (mins) - Descriptive Statistics:")
        print( cb_o_df[['Bike ID', 'Trip Duration (mins)', 'Idle Time (mins)']].describe() )
        
    except KeyError:
        print(f"> Output DataFrame with 'Idle Time' column was not properly created")

    # Write the processed DataFrame to a CSV file
    cb_p_o_file = "IdleTime-" + cb_p_i_file
    print(f"> Writing the processed data to the output file: { cb_p_o_file }")    
    %time cb_o_df.to_csv(cb_p_o_file, index=False)
    

 
*** PROCESSING FILE: new-Paths-JC-201902-citibike-tripdata.csv *****************************************
> Records in this file: 18565
Wall time: 3.61 s
> Idle Time (mins) - Descriptive Statistics:
            Bike ID  Trip Duration (mins)  Idle Time (mins)
count  18565.000000          18565.000000      18096.000000
mean   28303.983248             10.088993        813.913269
std     1556.916403            127.863561       1425.133492
min    26152.000000              1.016667          0.063083
25%    26267.000000              3.533333         89.935271
50%    29261.000000              5.000000        486.319650
75%    29513.000000              7.666667        866.402004
max    35652.000000           9528.450000      24973.685083
> Writing the processed data to the output file: IdleTime-new-Paths-JC-201902-citibike-tripdata.csv
Wall time: 670 ms
 
*** PROCESSING FILE: new-Paths-JC-201901-citibike-tripdata.csv *****************************************
> Records in this file: 19676
Wall 